# Install and import

In [1]:
!pip install mysql-connector
!pip install pymysql
!pip install ipython-sql
!pip install mysql-connector-python
!pip install sqlalchemy
!pip3 install skorch

# Data
from IPython.display import clear_output
import pandas as pd
import numpy as np
import math
from datetime import datetime
import matplotlib.pyplot as plt
from itertools import product

# Database
import sqlalchemy
import pymysql.cursors
import mysql
from mysql.connector import Error
from mysql.connector import errorcode
import mysql.connector

# System
import sys
import subprocess
import pkg_resources
import multiprocessing as mp
import warnings
import json 
from contextlib import redirect_stdout
from datetime import datetime, timedelta

# Neural network
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

clear_output()
warnings.filterwarnings(action='once')

# Config

In [2]:
CONFIG = {
        'host': '',
        'port': ,
        'user': '',
        'password': '',
        'database': '',
        'use_pure': 
    }

# Read database

In [3]:
def readDatabase(sql: str, columns) -> pd.DataFrame:
    # Read the database using the CONFIG variable and returns a Dataframe holding the data from the table
    conn = mysql.connector.connect(**CONFIG)
    cursor = conn.cursor()
    cursor.execute(sql)
    df = pd.DataFrame(cursor.fetchall(), columns = columns)
    conn.close()
    
    return df

def readColumns(sql: str) -> list:
    # Read the database using the CONFIG variable and returns a Dataframe holding the tables column names
    conn = mysql.connector.connect(**CONFIG)
    cursor = conn.cursor()
    cursor.execute(sql)
    columns = [column[0] for column in cursor.fetchall()]
    
    return columns

# Training the model

In [4]:
def trainModel(model_name: str, train_model: nn.Sequential, parameters: dict = None, validate: bool = True, shuffle = False):
    # Define default settings, that are replaced with GridSearchCS best_params_
    best_parameters = {'lr': 0.005, 'batch_size': 256, 'optimizer': None}
    # Database
    database_train = 'Train_Dynamo'
    database_test = 'Test_Dynamo'
    train_rows = 2_000_000
    test_rows = 1_000_000
    target_column = 'Energy_Usage'
    # Amount of epochs
    epochs = 5
    
    # Calculates haw many chunks is the data split during training
    chunks = math.ceil(1_000_000 / best_parameters['batch_size'])

    # Set the model to use GPU if available, else CPU
    model = train_model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Set the loss function and optimizer function
    criterion = nn.MSELoss()
    best_parameters['optimizer'] = torch.optim.Adagrad(model.parameters(), best_parameters['lr'])
    
    # Get the column names from the database
    columns = readColumns(f'DESC {database_train};')
    columns_test = readColumns(f'DESC {database_test};')

    # Run the GridSearchCV
    if parameters is not None:
        # Read data from the database
        df = readDatabase(f'SELECT * FROM {database_train} LIMIT {train_rows};', columns)
        # X is the feature used for training
        X = df.drop(['Energy_Usage'], axis = 1)
        X = torch.FloatTensor(X.values)
        # y is the target feature that the model is trying learn.
        y = df['Energy_Usage']
        y = torch.FloatTensor(y.values).reshape([-1, 1])
        best_parameters = gridSearchNN(model_name, model, parameters, X, y)
        best_parameters['optimizer'] = best_parameters['optimizer'](model.parameters(), best_parameters['lr'])

    start =  datetime.now()
    ###
    ### Define variables used for evaluating the model
    ###
    # Define variables used for evaluating learning
    running_loss = 0
    valid_loss = 0
    samples = 0
    losses = []
    validation_loss = []
    # Define variables used for calculating errors
    mse_list = []
    mae_list = []
    r2_list = []

    for epoch in range(epochs):
        # Define error and loss variables for epoch
        epoch_time =  datetime.now()
        epoch_loss = 0
        predicts = []
        labels = []

        # Read data from database
        partial_df = readDatabase(f'SELECT * FROM {database_train} LIMIT {train_rows};', columns)
        if shuffle:
            partial_df = partial_df.sample(frac=1)

        for chunk in range(chunks):
            # Checks if Dataframe is empty, to avoid errors
            if partial_df.shape[0] == 0:
                continue

            # Training the model
            model.train()
            best_parameters['optimizer'].zero_grad()
            # Read data from the database
            df = partial_df.iloc[best_parameters['batch_size'] * chunk:best_parameters['batch_size'] * chunk + best_parameters['batch_size']]
            # X is the feature used for training
            X = df.drop(['Energy_Usage'], axis = 1)
            X = torch.FloatTensor(X.values)
            # y is the target feature that the model is trying learn.
            y = df['Energy_Usage']
            y = torch.FloatTensor(y.values).reshape([-1, 1])

            # Getting the prediction and loss.
            # y_hat is the target prediction
            y_hat = model.forward(X.to(device))
            loss = criterion(y_hat, y.to(device))
            running_loss += loss
            epoch_loss += loss
            samples += 1

            loss.backward()
            best_parameters['optimizer'].step()

        # Calculate the loss and add it to the list
        losses.append(running_loss / samples)
    
        # Validate the model after each epoch
        if validate:
            model.eval()
            # Small memory manage action.
            del partial_df
            # No need for gradient calculation, since model isn't learning here.
            with torch.no_grad():
                # Read database and drop unnecessary column that was left behind.
                test_df = readDatabase(f'SELECT * FROM Test_Dynamo LIMIT {test_rows};', columns_test)
                #
                ###
                #### Remove if not necessary ###
                test_df = test_df.drop(columns="index", axis=1)
                ### ######################## ###
                ###
                #
                x = 0
                while x < test_rows:
                    # Read data from the database
                    partial_df = test_df.iloc[x:x+best_parameters['batch_size']]
                    # X is the feature used for training
                    X = partial_df.drop([target_column], axis = 1)
                    X = torch.FloatTensor(X.values)
                    # y is the target feature that the model is trying learn.
                    y = partial_df[target_column]
                    y = torch.FloatTensor(y.values).reshape([-1, 1])

                    # Getting the prediction and loss.
                    # y_hat is the target prediction
                    y_hat = model.forward(X.cuda())
                    loss = criterion(y_hat, y.cuda())
                    valid_loss += loss.item()
                    x += best_parameters['batch_size']

                    # Add the labels and prediction to list for later use.
                    for j in range(len(y)):
                        labels.append(y[j].item())
                        predicts.append(y_hat[j].item())

                    # Calulate errors
                    mse = F.mse_loss(y_hat, y.cuda())
                    mae = F.l1_loss(y_hat, y.cuda())
                    r2 = r2_score(y.cuda().cpu().numpy(), y_hat.cuda().cpu().numpy())
                    mse_list.append(mse)
                    mae_list.append(mae)
                    r2_list.append(r2)
                
                # Calculate validation loss and how many of the predictions are within acceptable range of the correct value.
                validation_loss.append(valid_loss/(epoch*test_rows+test_rows))
                threshold = len(compareResults(np.array(labels), np.array(predicts)))/len(labels)
                
        print(f'Time: {datetime.now() - epoch_time}\tAccuracy: {threshold*100}%\nTraining loss: {running_loss / samples}\tValidation loss: {valid_loss/(epoch*test_rows+test_rows)}\n')

    print((datetime.now() - start))
    # Create the name for the model and save it
    opt_name = best_parameters['optimizer']._zero_grad_profile_name.split('#')[-1].split('.')[0]
    lr = best_parameters['lr']
    batch_size = best_parameters['batch_size']
    model_name = f'{model_name}_{opt_name}_{lr}_{batch_size}'.replace('.', '')
    model_name += '.pt'
    torch.save(model, model_name)
    #error_dict = {'mse': , 'mse':}
    #errors = errorValues(labels, predicts, )
    return losses, validation_loss, [mse_list, mae_list, r2_list], model_name, threshold

# Compare results

In [5]:
def compareResults(label: np.array, pred: np.array, error_margin: float = 0.05) -> np.array:
    # Calculate the error percentage
    return pred[abs(pred/label - 1) <= error_margin]

In [6]:
def errorValues(labels: list, predicts: list, error_dict: dict) -> dict:

    error_list = np.absolute(labels - predicts)

    # Calculate if guesses are within 5% of the correct values
    Threshold_P = len(compareResults(labels, predicts))/len(labels)

    # Calculate error values
    Avg_Error = error_list.sum() / len(error_list)
    Min_Error = error_list.min()
    Max_Error = error_list.max()

    # Calculate losses
    MSE_Loss = (sum(error_dict['mse']) / len(error_dict['mse']))
    MAE_Loss = (sum(error_dict['mae']) / len(error_dict['mae']))

    # Calculate R2 score
    R2_Score = (sum(error_dict['r2']) / len(error_dict['r2']))
    
    errors = {'Threshold_P': Threshold_P, 'Avg_Error': Avg_Error, 'Min_Error': Min_Error, 'Max_Error': Max_Error, 'MSE_Loss': MSE_Loss.item(), 'MAE_Loss': MAE_Loss.item(), 'r2': R2_Score}
    
    return errors

# GridSearch

In [7]:
def gridSearchNN(name: str, model: nn.Sequential, parameters: dict, X: torch.FloatTensor, y: torch.FloatTensor, max_epochs: int = 10, cv: int = 3, scoring: str = 'r2', refit : bool = False, verbose: int = 3) -> dict:
    # Prepare the model for GridSearchCV
    net = NeuralNetRegressor(model,
                             max_epochs = max_epochs,
                             lr = 0.01,
                             verbose = verbose)
    # Create a filename
    time = str(datetime.now().time()).split('.')[0].replace(':', '')
    date = str(datetime.now().date()).replace('-', '')
    file_name = f'grid_log_{name}_{date}_{time}.txt'

    # Running the GridSearchCV and logging it to the file
    with open(file_name, 'w') as f:
        with redirect_stdout(f):
            print(name, '\n')
            gs = GridSearchCV(net, parameters, refit = refit, scoring = scoring, verbose = verbose, cv = cv)
            gs.fit(X, y)

    # Add additional information to the log
    results_text = '\n\nResults\n'
    for key, value in gs.cv_results_.items():
        results_text += f'{key}: {value}\n'
    with open(file_name, "a") as file:
        file.write(results_text)
        
    return gs.best_params_

# Neuron increments

In [8]:
# Simple means to increase/decrease the models neurons
def addNeurons(previous: int, increment: int) -> int:
    return previous + increment

def multiplyNeurons(previous: int, increment: int) -> int:
    return int(previous * increment)

# Model builder

In [9]:
def buildModel(input: int, output: int, inputs_output: int, method: str, layers: int, neuron_increment: int, ignore_ones = False) -> nn.Sequential:    
    # Contains all the neuron increment functions with a keyword to call them
    method_functions = {'add': addNeurons, 'mult': multiplyNeurons}
    
    # Checks if first layers neurons are greater than 0, If not, then uses increment function to set it
    if inputs_output <= 0:
        inputs_output = method_functions[method](input, neuron_increment)
        
    # Define the model and adds the input layer
    model = nn.Sequential(nn.Linear(input, inputs_output))
    # Keep track of the amount of neurons in previous layer
    previous = inputs_output

    # Build the model.
    for layer in range(0, layers-2):
        # Adds the activation function
        model.append(nn.ReLU())
        
        # Calculates the input and output neurons for the next layer
        next_neurons = method_functions[method](previous, neuron_increment)
        # Checks Special cases and interupts the run to inform the user.
        if next_neurons == 1 and not ignore_ones:
            raise ValueError('Number of neurons reached 1 before output. If this was by design, use ignore_ones = True to ignore this.')
        if next_neurons == 0:
            raise ValueError('Number of neurons reached 0.')
            
        # Adds the next layer
        model.append(nn.Linear(previous, next_neurons))
        # Update  the neuron amount
        previous = next_neurons
    
    # Adds the activation function and the output layer
    model.append(nn.ReLU())
    model.append(nn.Linear(previous, output))
    
    return model

# Main function for finding the hyperparameters for each model

In [10]:
def networkLooper(nn_parameters: dict, grid_parameters: dict = None) -> None:
    # Split the dictionary to variables to make it more readable
    input = nn_parameters['input']
    output = nn_parameters['output']
    inputs_output = nn_parameters['inputs_output']
    
    methods = nn_parameters['method']    
    layers = nn_parameters['layers']
    neuron_increment = nn_parameters['neuron_increment']
    
    # Checks that variables are in list
    if type(methods) != list:
        methods = [methods]
    if type(layers) != list:
        layers = [layers]
    if type(neuron_increment) != list:
        neuron_increment = [neuron_increment]
    
    # Define the file name
    time = str(datetime.now().time()).split('.')[0].replace(':', '')
    date = str(datetime.now().date()).replace('-', '')
    file_name = f'Log_{date}_{time}.txt'
    with open(file_name, 'w') as file:
        file.write('')
        
    # Creates a single list holding all the possible parameter variations for the model by using Cartesian Product
    all_list = [methods, layers, neuron_increment]
    param_combination = product(*all_list)

    # Buildin a model, trainng it and logging the results
    for param in param_combination:
        # param[0] = method, param[1] = layers,param[2] = neurons
        # Building the model itself and giving it a name
        model = buildModel(input, output, inputs_output, *param)
        name = f'{inputs_output}_{param[0]}_{param[1]}_{param[2]}'
        print(name)
        # Training the model
        losses, validation_loss, errors, name, threshold = trainModel(name, model, grid_parameters)
        # Preparing the losses for the file
        loss_log = ''
        val_log = ''
        for x in range(len(losses)):
            loss_log += f'{losses[x]}\n'
            val_log += f'{validation_loss[x]}\n'
        print('---------------')
        # Writing the model results to the log
        log = f'{name}\nInput: {input}\nOutput: {output}\nInput layers output: {inputs_output}\nLayers: {param[1]}\nNeuron increment:\n\ttype: {param[0]}\n\tincrement : {param[2]}\nLosses:\n{loss_log}\n'
        log += f'Validation loss:\n{val_log}\nErrors:\n\tmse: {sum(errors[0]) / len(errors[0])}\n\tmae: {sum(errors[1]) / len(errors[1])}\n\tr2: {sum(errors[2]) / len(errors[2])}\nAccuracy: {threshold*100}%\n\n'
        with open(file_name, 'a') as file:
            file.write(log)

# Defining the parameters for testing

In [11]:
# layers value includes the input layer, hidden layers and the output layers.
# input is the number of neurons for the input layers input. This is also the number of features used for training
# output is the number of neurons for the output layers output. This is regression model, so the output is 1
# neuron_increment is the value used to increase or decrease the number of neurons between layers
# method is the key used to call the function
# inputs_output is the first layers inputs output. This is used to make is easier to plan how the number of neurons change between layers
nn_parameters = {'layers': [3], 'input': 31, 'output': 1, 'neuron_increment': [-10], 'method': 'add', 'inputs_output': 30}
# Define the hyperparameters for GridSearchCV
grid_parameters = {'batch_size': [256], 'optimizer': [torch.optim.Adagrad, torch.optim.SGD], 'lr': [0.01]}

In [12]:
networkLooper(nn_parameters, grid_parameters)

30_add_3_-10
Time: 0:03:06.328108	Accuracy: 5.8404%
Training loss: 0.10051791369915009	Validation loss: 0.0007154945546785602

Time: 0:02:59.088400	Accuracy: 6.6952%
Training loss: 0.07258964329957962	Validation loss: 0.0005996980940471985

Time: 0:03:04.550815	Accuracy: 13.944899999999999%
Training loss: 0.0613241009414196	Validation loss: 0.0005204542587328858

Time: 0:03:01.137488	Accuracy: 18.5348%
Training loss: 0.05526987463235855	Validation loss: 0.00047312659626784444

Time: 0:02:58.599494	Accuracy: 21.6815%
Training loss: 0.05147917941212654	Validation loss: 0.0004427914728561766

0:15:09.712451
---------------
